In [45]:
import sys
import itertools
import re
from collections.abc import Sequence
from pprint import pprint

path_to_fasta_file = '/home/alakhani/Coursera-Projects/PythonforGenomicDataScience/FASTA_Files/dna.example_copy.fasta'

In [ ]:
# FASTA record descriptions begin with '>', 
# Simplest way of counting without doing anything else:

with open(path_to_fasta_file,'r') as fasta_file:
    count = fasta_file.read().count('>')

# print(count)


In [ ]:
# Dictionary comprehension directly from file looks cool but
# does not seem amenable for collecting sequence values in the same iteration.

with open(path_to_fasta_file,'r') as fasta_file:
    seq_dict = {line.strip('>\n'): None for line in fasta_file 
                if line[0] == ">"}
    
# print('Length of dictionary made "the python way"', len(seq_dict_2),'\n')
# pprint(seq_dict.items())


## Collecting sequences from a FASTA file into a dictionary

### The more familiar, "less pythonic" way:

In [ ]:
def fasta_todict(filepath):
    with open(filepath,'r') as fasta_file:
        header = ""
        seq_accumulator = []
        sequences = {}
        for line in fasta_file:
            # For first line/header.
            if line[0] == ">" and not (header or seq_accumulator):     
                header = line.strip('>\n')
            elif line[0] == ">" and seq_accumulator:
                sequences[header] = ''.join(seq_accumulator)
                seq_accumulator.clear()
                header = line.strip('>\n')
            elif header:
                seq_accumulator.append(line.strip())
            else:
                print("Check file."
                    "Missing header on first line or two consecutive headers.")
        # Loop ends before the last sequence is paired with header.
        sequences[header] = ''.join(seq_accumulator)
        return sequences

# seqs = fasta_todict(path_to_fasta_file)    
# print('Length of sequence dictionary"', len(sequences))
# print(sequences)
# partial_list = [item[1] for item in enumerate(seqs.items()) if item[0] < 4]
# z = iter(seqs.items())
# partial_list = [next(z) for _ in range(4)]
# print(list(seqs.items())[:4])
# print(partial_list)

## That was clunky. Is there a cleaner way with itertools?

Iterators save memory with 'lazy execution', i.e. they don't start doing the thing until you ask them.
This will let us directly feed keys and values into a dictionary without reading into a list first.  \*\*\*\*

We can use a key function to tell `groupby()` to split the file into chunks of consecutive lines that either do or do not start with '>'.
`groupby()` returns tuples of `(bool, _grouper)`. `__grouper` is not the sequence chunk, it is itself an iterator that "knows" to go through the file and collect lines into chunks, when or if we ask.

We need to iterate through (call `__next__` on) the second item in each tuple to tell `__grouper` to actually start iterating through the file and retrieve the next chunk of lines.

Calling `next()` retrieves and 'expends' the next value(s) in an iterator (same thing a for loop does), so if `next()` is used in a loop those values or 'positions in the list' will get 'skipped' in the next loop.

\*\*\*\* This was my mistake! Groupby is more memory efficient than reading in the whole file at once and then making a second temporary list. But groupby is still constructing a temporary list of lines for each chunk under the hood, then we're iterating over those temporary lists to join lines. I.e., the same thing we did above.  

In [ ]:
def joinlines(iterable):
    return ''.join(line.strip() for line in iterable)

def fasta_todict2(filepath):
    with open(filepath,'r') as fasta_file:
        if fasta_file.readline()[0] != '>':
            print("File needs to start with header preceded by '>'.") 
        else:
            fasta_file.seek(0)
            chunks = itertools.groupby(fasta_file, key=lambda x: x[0]=='>')
            seqdict = {joinlines(chunk[1]).strip('>'): 
                       joinlines(next(chunks)[1]) for chunk in chunks}
            return seqdict

# seqdict = fasta_todict2(path_to_fasta_file)

# print(list(seqdict.items())[:4])

In [ ]:
# Similar to next(), we can zip an iterator with itself.
# This will also 'expend' multiple values in one iteration over the zip object.
# Note: zip() in Python 3 behaves the same as itertools.izip(). 
#       zip() in Python 2 returns the entire list at once.  
# Also, zipping with the iterator lead to unexpected behavior with the headers 
# being dropped. Had to turn chunks into a generator to work as expected.
#
# Zipping is unnecessary for this application since 
# we're generating tuples just to unpack them.


def fasta_todict3(filepath):
    with open(filepath,'r') as fasta_file:
        if fasta_file.readline()[0] != '>':
            print("File needs to start with header preceded by '>'.") 
        else:
            fasta_file.seek(0)
            chunks = (joinlines(chunk[1]).strip('>') for chunk in 
                      itertools.groupby(fasta_file, key=lambda x: x[0]=='>'))
            seqdict = {chunk1: chunk2 for chunk1, chunk2 in zip(chunks,chunks)}
            return seqdict

# seqs = fasta_todict3(path_to_fasta_file)

# print(list(seqs.items())[:4])

### **Proof that all three methods yield equivalent dictionaries.**

In [ ]:
seqs1 = fasta_todict(path_to_fasta_file)
seqs2 = fasta_todict2(path_to_fasta_file)
seqs3 = fasta_todict3(path_to_fasta_file)

seqs1 == seqs2 == seqs3

### **Did that optimization make any difference for this 59 kb file?**

In [ ]:
from timeit import timeit

timeit("seqs1 = fasta_todict(path_to_fasta_file)", 
       "path_to_fasta_file = '/home/alakhani/Coursera-Projects/PythonforGenomicDataScience/FASTA_Files/dna.example_copy.fasta'\n"
"""def fasta_todict(filepath):
    with open(filepath,'r') as fasta_file:
        header = ""
        seq_accumulator = []
        sequences = {}
        for line in fasta_file:
            # For first line/header.
            if line[0] == ">" and not (header or seq_accumulator):
                header = line.strip()
            elif line[0] == ">" and seq_accumulator:
                sequences[header] = ''.join(seq_accumulator)
                seq_accumulator.clear()
                header = line.strip()
            elif header:
                seq_accumulator.append(line.strip())
            else:
                print("Check file.")
        # Loop ends before the last sequence is paired with header.
        sequences[header] = ''.join(seq_accumulator)
        return sequences""",
        number=10000)

In [ ]:
from timeit import timeit

timeit("seqs2 = fasta_todict2(path_to_fasta_file)", 
       "path_to_fasta_file = '/home/alakhani/Coursera-Projects/PythonforGenomicDataScience/FASTA_Files/dna.example_copy.fasta'\n"
"""def joinlines(iterable):
    return ''.join(line.strip() for line in iterable)
def fasta_todict2(filepath):
    with open(filepath,'r') as fasta_file:
        if fasta_file.readline()[0] != '>':
            print("File needs to start with header preceded by '>'.") 
        else:
            fasta_file.seek(0)
            chunks = itertools.groupby(fasta_file, key=lambda x: x[0]=='>')
            seqdict = {joinlines(chunk[1]): joinlines(next(chunks)[1]) for 
                       chunk in chunks}
            return seqdict""",
        number=10000)

In [ ]:
from timeit import timeit

timeit("seqs3 = fasta_todict3(path_to_fasta_file)", 
       "path_to_fasta_file = '/home/alakhani/Coursera-Projects/PythonforGenomicDataScience/FASTA_Files/dna.example_copy.fasta'\n"
"""def joinlines(iterable):
    return ''.join(line.strip() for line in iterable)
def fasta_todict3(filepath):
    with open(filepath,'r') as fasta_file:
        if fasta_file.readline()[0] != '>':
            print("File needs to start with header preceded by '>'.") 
        else:
            fasta_file.seek(0)
            chunks = (joinlines(chunk[1]) for chunk in 
                      itertools.groupby(fasta_file, key=lambda x: x[0]=='>'))
            seqdict = {chunk1: chunk2 for chunk1, chunk2 in zip(chunks,chunks)}
            return seqdict""",
        number=10000)

## Lol
Indeed, all that glitters is not gold. The 'dumb way' was faster and will be more readable (to me) in 6 months.

It seems the extra overhead from creating extra intermediate objects to perform disjointed nested loops, plus calling a lambda function on each line instead of a simple conditional outweighed the idea that "it's faster because it's implemented in C".

To be fair, these tools seem more oriented towards managing memory consumption, which is contradicted by copying everything into a dictionary anyway. 
If we truly needed to avoid copying data unless or until it was needed, it could make more sense to save the `__grouper` objects as dictionary values. However, since the philosophy of Python is to abstract away these types of concerns, maybe it would be unnecessary even then.
___

## Let's play with dictionaries some more

First we'll define `fasta_to_nesteddict` to create a nested dict, so we can associate additional properties with each sequence name.

Then we'll pretend we already have a flat dictionary and can't or don't want to create a copycat function. In this case we need to replace dictionary values with a nested dictionary, while retaining the original dictionary value somewhere. 

#### This highlights one of the advantages of implementing a class rather than a dictionary. What if we want to say, store a property of a property? Class implementation will be explored later.

In [ ]:
def fasta_to_nesteddict(filepath):
    '''Returns {$FASTA_HEADER: {'Sequence': $SEQUENCE}}'''
    with open(filepath,'r') as fasta_file:
        header = ""
        seq_accumulator = []
        sequences = {}
        for line in fasta_file:
            # For first line/header.
            if line[0] == ">" and not (header or seq_accumulator):     
                header = line.strip('>\n')
            elif line[0] == ">" and seq_accumulator:
                sequences[header] = {'Sequence': ''.join(seq_accumulator)}
                seq_accumulator.clear()
                header = line.strip('>\n')
            elif header:
                seq_accumulator.append(line.strip())
            else:
                print("Check file."
                    "Missing header on first line or two consecutive headers.")
        # Loop ends before the last sequence is paired with header.
        sequences[header] = {'Sequence': ''.join(seq_accumulator)}
        return sequences
    
# nestedseqdict = fasta_to_nesteddict(path_to_fasta_file)
# nestedseqdict_items = iter(nestedseqdict.items())
# print([next(nestedseqdict_items) for _ in range(4)])

## Adding another level of hierarchy to a flat dictionary 

In the next function, two parameters are given default values of None, when the intended behavior is to have these parameters default to empty lists. 

This is due to Python's treatment of mutable default arguments:

> <https://stackoverflow.com/questions/1132941/least-astonishment-and-the-mutable-default-argument>

> <https://docs.python-guide.org/writing/gotchas/#mutable-default-arguments>

When functions are defined with default values in Python, objects are initialized with these default values. These default argument *objects*, not the defined default *values*, "stick with" the function object throughout the program. 

If an argument with a mutable default is mutated within a function, calling the function without explicitly declaring this argument will change the argument's "default value" for future function calls.
This implies a defensive programming practice of explicitly defining mutable arguments if we intend them to be empty.

In the function below we do not mutate addkeys or addvals, but it is advisable to avoid mutable defaults as a rule unless this state dependence is specifically intended.

In [ ]:
# This function requires that addvals support indexing.

def nestdict(currentdict, newvalname="Old val", addkeys=None, 
             addvals: Sequence=None):
    '''Replaces dict values with {newvalname: currentdict[key]}. 

    New key:val pairs can be added to each key's nested dictionary using addkeys and addvals.
    Members of addvals can be called on currentdict values.
    If addkeys is longer than addvals, unmatched members of addkeys are initialized to None.
    Unmatched members of addvals are ignored. 
    '''
    newdict = {}
    addkeys = [] if addkeys is None else addkeys
    addvals = [] if addvals is None else addvals 
    for key, val in currentdict.items():
        lvl2dict = {}
        lvl2dict[newvalname] = val
        for index, name in enumerate(addkeys):
            try:
                x = addvals[index]
            except IndexError:
                x = None
            except:
                print("addvals in nestdict must be None or a Sequence.")
                raise
            if callable(x):
                lvl2dict[name] = x(val)
            else:
                lvl2dict[name] = x
        newdict[key] = lvl2dict
    if len(addvals) > index + 1:
        print("More values than keys provided, excess values ignored.")
    return newdict

# z = nestdict(fasta_todict(path_to_fasta_file), 'sequence', ['length','ORFs'], [lambda x: len(x), None])
# z_items = iter(z.items())
# print([next(z_items) for _ in range(4)])

**Another approach**

`zip_longest()` is cleaner and allows compatibility with iterables that do not support indexing, although the latter seems to have limited benefit. 

Similar to the previous version, if addvals is longer than addkeys for some reason, key values will default to `None` once addkeys is exhausted.  
`zip_longest()` will then generate tuples with the pattern `((None, val_n-2), (None, val_n-1)...(None, val_n))`, each excess value being overwritten until only the last value in addvals is saved.  
Managing this will be left to the user. A warning cannot be based on `len` in this case because at no point is there a guarantee that addvals supports `len`.  

\#whenthedocstringisaslongasthefunction

In [ ]:
def nestdict(currentdict, newvalname="Old val", addkeys=None, addvals=None):
    '''Replaces dict values with {newvalname: currentdict[key]}. 

    addkeys and addvals are zipped to generate new key:val pairs for each key's nested dictionary. 
    Members of addvals can be called on currentdict values.
    If addkeys is longer than addvals, unmatched members of addkeys are initialized to None.
    If addvals is longer than addkeys, unmatched members of addvals will be lost except for the last element. The last members of the nested dicts will be (None: addvals[n]).
    '''
    newdict = {}
    addkeys = [] if addkeys is None else addkeys
    addvals = [] if addvals is None else addvals
    for key, val in currentdict.items():
        lvl2dict = {}
        lvl2dict[newvalname] = val
        lvl2dict.update((k, v(val)) if callable(v) else (k, v)
                        for k, v in 
                        itertools.zip_longest(addkeys, addvals))
        newdict[key] = lvl2dict
    return newdict

# z = nestdict(fasta_todict(path_to_fasta_file), 'sequence', ['length','ORFs', 'more', 'keys'], {'dictionary': 'inception', 'this':'is', 'why':'classes', 'are':'probably', 'better': None}.items())
# z_items = iter(z.items())
# print([next(z_items) for _ in range(4)])

In [ ]:
seqdict = fasta_todict(path_to_fasta_file)
nestedseqdict = fasta_to_nesteddict(path_to_fasta_file)

In [ ]:
s = [i for i in seqdict.values()]
n = [i['Sequence'] for i in nestedseqdict.values()]
s == n

### Well now what if we want to add more values to the nested dictionaries? <sup><sup><sup>or add values to nested nested dictionaries?</sup></sup></sup>

\#needasequenceclass

In [ ]:
# Object-oriented, modifies dictionary directly.

def update_ndicts_inplace(currentdict, addkeys=None, addvals=None, modkey=''):
    '''Members of addvals can optionally be called on nesteddict[modkey].
    Uses zip_longest(addkeys, addvals).
    '''
    addkeys = [] if addkeys is None else addkeys
    addvals = [] if addvals is None else addvals
    if modkey:
        for key, ndict in currentdict.items():
            ndict.update((k, v(ndict[modkey])) if callable(v) 
                        else (k, v) 
                        for k, v in itertools.zip_longest(addkeys, addvals))
    else:
        for key, ndict in currentdict.items():
            ndict.update((k, v) 
                        for k, v in itertools.zip_longest(addkeys, addvals))

In [ ]:
# Pure function, returns a new dictionary without modifying the original.
from copy import deepcopy

def updated_ndict(currentdict, addkeys=None, addvals=None, modkey=''):
    '''Members of addvals can be called on nesteddict[modkey].
    Uses zip_longest(addkeys, addvals).
    '''
    addkeys = [] if addkeys is None else addkeys
    addvals = [] if addvals is None else addvals
    dictcopy = deepcopy(currentdict)
    if modkey:
        for key, ndict in dictcopy.items():
            ndict.update((k, v(ndict[modkey])) if callable(v) 
                         else (k, v) 
                         for k, v in itertools.zip_longest(addkeys, addvals))
    else:
        for key, ndict in dictcopy.items():
            ndict.update((k, v) 
                        for k, v in itertools.zip_longest(addkeys, addvals))
    return dictcopy

### Finding ORFs

#### The power of regex

ORFs are delimited by a start codon ('ATG' in the case of DNA and 'UAG' in the case of RNA), and an "in-frame" stop codon. *Typically* in DNA and RNA, these correspond to 'TAA', 'TAG', and 'TGA', or 'UAA', 'UAG', and 'UGA' respectively. "In-frame" means the number of bases between the stop and start codons is a multiple of 3. 

'ATGATAA' contains two codons (groups of three bases): 'ATG' and 'ATA'. Even though 'TAA' is present in the sequence, it is not recognized as a stop codon because the number of intervening nucleotides is not a multiple of 3. 

One way to find an ORF is to find the first instance of 'ATG', str.find() instances of 'TAA', 'TAG', and 'TGA' following the start codon. The first tricky bit is that str.find() only searches for a particular sequence and only returns the index of the first location, so we would need separate iterations for each stop codon sequence.

We would stop iterating at the first instance where the number of nucleotides between the start of the start codon and the start of the stop codon is a multiple of 3 (n % 3 == 0). 

Regular expressions can express this problem concisely, in an "ancient magic" sort of way. This comes at the price of readability and speed: regex can be more than an order of magnitude slower than built-ins: <https://stackoverflow.com/questions/8958229/is-a-regular-expression-faster-than-using-replace>. However if you want to show how `1337` you are and like Googling how your own code works when you have to look at it six months later, regex is the way to go.

Quick breakdown of the expression used below:

- Expressions in parenthesis indicate logical separations. `(ATG)` means look for the first instance of the sequence `ATG`. 
  - Square brackets `[ATG]` would just look for the first instance of 'A' and/or 'T' and/or 'G'. Any of these characters in any order would be sufficient to start the next stage of the search.
- `(\S{3})`: `\S` (not to be confused with `\s`!) is a wildcard for any non-whitespace character. The slash needs to be escaped unless using a raw string. 
  - Braces `{}` are notation for repetition: `{3}` means repeats of 3. Regex also supports looking for a range of repeats, such as `{min,[max]}`.
  - The `\s` could also be replaced by sequences such as `[ATGUCatguc]` or `[A-Z,a-z]`, but DNA and RNA sequences sometimes contain whitespaces between codons.  
- `*` means "zero or more instances". Combined with the previous expression, `(.{3})*` means "look for one or more repeats of 3.
- `*` uses "greedy" evaluation, meaning it will keep going as far as it can and then work backwards to fulfill the rest of the conditions. This yields the maximum number of the preceding expression that still fulfills the other conditions for the expression. So if the expression could be satisfied by a sequence containing 5 or 50 repeats of 3, greedy evaluation would return the sequence containing 50 repeats. This is not what we want! We don't want to read through valid stop codons, we want to stop at stop codons.
- `*?` reigns in `*` and switches to lazy evaluation. This means it stops once it finds an expression that satisfies the necessary conditions. This is the intended behavior here. 
 - `??` is super lazy, and looks for the lesser of 0 or 1 of the previous clause.

Regex is like a very sharp knife: efficient in the hands of an experienced user, but new users need to take extra care to avoid mistakes.

In [ ]:
import re
s1 = 'AYGCTATGCGAGCTASYUYGCTAGUTAATGAYSGC'
s2 = 'AYGCTATGCGAGCTAAGATGCGGTAGTAATTTGAYSGC'
s3 = 'AYGCTATGCGAGCTGATGCGGTAGTAATTTGAYSGC'

In [ ]:
m1 = re.search(r'(ATG)(\S{3})*(TAG|TAA|TGA)',s1)
m2a, m2b, m2c = (re.search(r'(ATG)(\S{3})*(TAG|TAA|TGA)',s2), 
                 re.search(r'(ATG)(\S{3})*?(TAG|TAA|TGA)',s2),
                 re.search(r'(ATG)(\S{3})??(TAG|TAA|TGA)',s2))
m3a, m3b = (re.search(r'(ATG)(\S{3})*?(TAG|TAA|TGA)',s3),
            re.search(r'(ATG)(\S{3})??(TAG|TAA|TGA)',s3))
print(f'm1: {m1}\nm2a: {m2a}\nm2b: {m2b}\nm2c: {m2c}\nm3a: {m3a}\nm3b: {m3b}')

Now we know how to find the first ORF in a sequence, which I'm defining as the substring containing the first start codon that is followed by an in-frame stop codon. 

In the example above, m3a is the first ORF even though the sequence of m3b ends first.

If we want to find all ORFs, we can slice the string after the start codon of the previous ORF and repeat the search process until re.search() returns `None`. 

`str[re.Match.start()+3:]` can be used to get the next slice, but the start and end positions of the next ORF will be reported relative to the beginning of the slice, not the original string. The actual index of `re.Match.start()` would need to be saved somewhere if we want to know the position of each ORF in the original string.

In [ ]:
m3c = re.search(r'((ATG)(\S{3})*?(TAG|TAA|TGA))',s3[m3a.start()+3:])
print(f'm3c: {m3c}')

Things get more tricky if we want to limit our search to a particular reading frame, and thorough testing is needed to make sure we're the behavior is what is intended. The idea is to look for a start codon that is in frame with the beginning of the string, i.e. it's preceded by groups of 3 or its index is divisible by 3. 

If we just add `(.{3})*?` to the beginning of the expression used above, we just collect all the triplets that precede the first ORF, even if the resulting sequence is out of frame. 

Adding `^` anchors the expression to the beginning of the string, meaning the triplets have to start from the beginning.

Lastly, this strategy returns extraneous codons that precede the ORF. Thankfully re.Match objects save the expressions that were returned by the individual "logical" parts of the search pattern. Surrounding the expression that represents the ORF with parentheses `((ATG)(.{3})*?(TAG|TAA|TGA))` will make it easy to extract without the irrelevant portion of the sequence, and parenthesizing the "triplet tracker" `((.{3})*)` will help us get an accurate span for the ORF. Otherwise only the penultimate triplet gets saved.

In [ ]:
import re
s4 = 'AYAGCTATGCGAATGGCTGATGCGGTAGTAATCTGAYSGC'

m4a = re.search(r'^((\S{3})*)((ATG)(\S{3})*?(TAG|TAA|TGA))', s4)
m4b = re.search(r'^((\S{3})*?)((ATG)(\S{3})*?(TAG|TAA|TGA))', s4)
m4c = re.search(r'^((\S{3})*)((ATG)(\S{3})*?(TAG|TAA|TGA))', s4[1:])
print(f'm4a: {m4a}\nm4b: {m4b}\nm4c: {m4c}')

In [ ]:
(m4a.groups() if m4a else None, 
m4b.groups() if m4b else None, 
m4c.groups() if m4c else None)

This raises the possibility of using an fstring to modify the search start position instead of slices. With the fstring approach, keep in mind that the curly braces in the regex need to be escaped by doubling them: `{{}}`. Triple braces are needed if we are interpolating a string within escaped braces.

Also, fstrings don't allow backslashes in any form. We could replace `\S` with `.` to accept any character including whitespaces, create a variable to interpolate, or concatenate. 

Parsing the combination of single, double, and triple braces in this expression turns into bit of a mess. Regex is less concise on net because either way some `current_index` needs to be updated with `+= len(str) + 3`.

In [ ]:
s = '\S'
i = 0
m4a = re.search(f'^({s}{{{i}}}({s}{{3}})*)((ATG)({s}{{3}})*?(TAG|TAA|TGA))', s4)
i = 1
m4b = re.search(f'^({s}{{{i}}}({s}{{3}})*)((ATG)({s}{{3}})*?(TAG|TAA|TGA))', s4)
i = 2
m4c = re.search(f'^({s}{{{i}}}({s}{{3}})*)((ATG)({s}{{3}})*?(TAG|TAA|TGA))', s4)
print(f'm4a: {m4a}\nm4b: {m4b}\nm4c: {m4c}')

In [ ]:
from typing import NamedTuple, Iterable
from collections import namedtuple
import re

# Not mutable so all desired attributes need to be set at once.
# More prelude to turning everything into a class a la BioPython...

orf = namedtuple('ORF', 
                 names:=['sequence', 'start', 'end', 'length', 'readingframe'],
                 defaults=[None for _ in range(len(names))])

# Helper function to retrieve one ORF from any frame.
# def getorf(seq: str, pattern: re.Pattern, startpos=0, endpos: int=...) -> orf:
#     if match := pattern.search(seq, 
#                                startpos, 
#                                endpos if endpos is not ... else len(seq)):
#         sequence = match.groupdict()['orf']
#         # 'start' is the index and 'end' is the "character number" index + 1, 
#         # so they can be used directly for slicing.
#         start = startpos + len(match.groupdict()['prior'])
#         length = len(sequence)
#         end = start + length
#         readingframe = start % 3
#         return orf(sequence, start, end, length, readingframe)
#     else:
#         return None

def getorfs_re(seq: str, startpos: int = 0, endpos: int = ..., 
        frame: int = ..., seqtype = 'DNA', stopcodons = None) -> list[orf]:
    '''Any character is allowed between start and stop codons.
    Reads to end of string if 'endpos' is None.
    'frame' can be any int < (len(seq) - startpos) and is interpreted relative to 'startpos'. 
    (startpos=x, frame=y) is equivalent to (startpos=x+y, frame=0) and 
    (startpos=0, frame=x+y).
    Assumes standard stop codons but an alternative set of stop codons may be used.
    Results sorted by start position.
    Reports start as index and end as index + 1 so they can be used directly
    in slices or range().
    '''
    startpos = startpos if frame is ... else startpos + frame
    endpos = len(seq) if endpos is ... else endpos
    stopcodons = '|'.join(stopcodons if stopcodons is not None
        else ['UAG','UGA','UAA'] if seqtype == 'RNA' or seqtype == 'rna' 
        else ['TAG','TGA','TAA']) 
    orfs = []

    # triplet searches must be lazy
    s = '\S'
    pattern = re.compile(f"(?P<prior>^{f'{s}*?' if frame is ... else '(.{3})*?'})"
                        +f"(?P<orf>(ATG)(.{{3}})*?({stopcodons}))", re.I)
    while (nextorf := pattern.search(seq[startpos:endpos])):
        sequence = nextorf.groupdict()['orf']
        # 'start' is the index and 'end' is the "character number" index + 1, 
        # so they can be used directly for slicing.
        start = startpos + len(nextorf.groupdict()['prior'])
        length = len(sequence)
        end = start + length
        readingframe = start % 3 + 1
        orfs.append(orf(sequence, start+1, end, length, readingframe))
        startpos = start + 3
    return orfs

#### That was a nightmare to debug...
and I ended up using slices anyway. The easiest thing would be to cheat and find all ORFs, then just filter the ones that are not in frame. 

Another KISS approach would be to collect the indices of start and stop codons in separate lists, then evaluate whether the distances between stop and start codons are divisible by 3 and return slices containing valid start-stop pairs. 

While iterating through the list of stop codons for each start codon is exponential behavior, it is way more efficient than a) the regex search method or b) looking for a start codon then looking for a stop codon, then going back to find the next start codon and looking for its matching stop codon, etc.
Some further optimization could be done to avoid iterating through stop codons that are known to be before the current start codon, but that is really not necessary for most cases. 

In [ ]:
from typing import NamedTuple, Iterable
from collections import namedtuple
import re

# Not mutable so all desired attributes need to be set at once.
# More prelude to turning everything into a class a la BioPython...

orf = namedtuple('ORF', 
                 names:=['sequence', 'start', 'end', 'length', 'readingframe'],
                 defaults=[None for _ in range(len(names))])

def getorfs(seq: str, startpos: int = 1, endpos: int = ..., frame: int = ...,
            seqtype = 'DNA', stopcodons: Iterable = ...) -> list[orf]:
    '''
    Arguments and return values are indexed to 1. #don'tblameme
    Any character is allowed between start and stop codons.
    Search is case-insensitive but return values preserve case.
    'frame' is interpreted relative to 'startpos' with an index of 1.
    (startpos=x, frame=y) is equivalent to (startpos=y, frame=x).
    Reads to end of string if 'endpos' is None.
    Assumes standard stop codons but an alternative set of stop codons may be used.
    Results sorted by start position.
    Reports start as index and end as index + 1 so they can be used directly
    in slices or range().
    '''
    startpos = startpos - 1 if frame is ... else startpos + frame - 2
    endpos = len(seq) if endpos is ... else endpos
    stopcodons = (stopcodons.upper() if stopcodons is not ... 
        else ['UAG','UGA','UAA'] if seqtype.upper() == 'RNA'
        else ['TAG','TGA','TAA'])
    orfs = []
    raised = seq.upper()
    starts = [i for i in range(startpos, endpos-3) 
              if raised[i:i+3] == 'ATG' 
              and (True if frame is ... else (i - startpos) % 3 == 0)]
    stops = [i for i in range(len(seq)-2)       # len-2 keeps seq[-3] in range.
             if (codon := raised[i:i+3])
             and any(codon == stop for stop in stopcodons) 
             and (True if frame is ... else (i - startpos) % 3 == 0)]
    for start in starts:
        for stop in stops:
            if start < stop and ((stop:= stop+3) - start) % 3 == 0:
               orfs.append(orf(seq[start:stop], 
                               start+1, stop, stop-start, start%3+1))
               break
    return orfs

In [ ]:
print((getorfs_re(s4), getorfs_re(s4,frame=1), getorfs_re(s4,frame=7)) == 
    (getorfs(s4), getorfs(s4,frame=2), getorfs(s4,frame=8)),'\n')

print(getorfs(s4), getorfs(s4,startpos=20), getorfs(s4,frame=7),sep='\n\n')

In [ ]:
from operator import attrgetter
from collections import defaultdict
from itertools import takewhile

nestedseqdict = fasta_to_nesteddict(path_to_fasta_file)
newseqdict = updated_ndict(
    nestedseqdict,['Length', 'ORFs'],[len, getorfs], modkey='Sequence')
iter1 = iter(nestedseqdict.items())
iter2 = iter(newseqdict.items())
# pprint([next(iter1) for _ in range(4)])
# pprint([next(iter2) for _ in range(4)])
orfsbysize = defaultdict(list)
for header, seq in newseqdict.items():
    seq['ORFs'].sort(key=attrgetter('length'))
    shortest = seq['ORFs'][0].length
    longest = seq['ORFs'][-1].length
    # use iterator to collect all sequences with the min or max length
    orfsbysize[shortest].extend([
        (header, shortorf) for shortorf in 
        takewhile(lambda x: x.length == shortest, seq['ORFs'])
        ])
    orfsbysize[longest].extend([
        (header, longorf) for longorf in 
        takewhile(lambda x: x.length == longest, seq['ORFs'][::-1])
        ])    

orfsbysize = sorted(orfsbysize.items())

### Looking for repeats of size N

Tempting to try regex again like `(.{N}){2,}`. We've seen before that regex can be unwieldy and at times difficult to predict, but it's so powerful once the magic expression is found. Here we want to be able to detect overlapping, consecutive, and non-consecutive repeats, and count their frequency. `str.count()` does not count overlapping occurrences. It would also be nice to save the locations of the repeats but first things first: identifying repeated sequences. 

In [ ]:
s5 = 'agacagacagacagt'
s6 = 'agacagacjhgfcjhfgacagacagt'
s7 = 'agacagacjhgfcjhfgacagatcjh'
n = 4
s = '\S'
# `?=` means lookahead without consuming the string so we can find overlapping matches
# matches any group of n non-whitespace characters 
expr = re.compile(f'(?=(?P<repeat>{s}{{{n}}}))')
# Matches groups of n non-whitespace characters that are later repeated. 
# Need to use +? so next repeat starts at least one char from beginning of first repeat.
# Resulting list contains duplicates of any group that contains >1 repeat. 
# Any pair of repeats is a match since the string is not consumed by lookahead.
expr = re.compile(f'(?=(?P<repeat>{s}{{{n}}}))(?={s}+?(?P=repeat))')
# Changing to greedy evaluation of spacers does not change this, 
# nor does matching repeats of the entire pattern, because these do not change
# the validity of any other combination of repeats. 
# expr = re.compile(f'((?=(?P<repeat>{s}{{{n}}}))(?={s}+(?P=repeat)))+')

# Combining the caret with lookahead was not straightforward. 
# '(\S{3})*?' can be used to match repeats that are in frame with each other,
# but it is not clear whether '^(\S{3})*?' can be used to match repeats that 
# are in frame with the start of the string.
# expr = re.compile(f'^(?=(?:{s}{{3}})*?(?P<repeat>{s}{{{n}}}))(?=(?:{s}{{3}})+?(?P=repeat))')
# expr = re.compile(f'(?=^(?:{s}{{3}})*?(?P<repeat>{s}{{{n}}}))(?=(?:{s}{{3}})+?(?P=repeat))')

expr.findall(s7)

### Named tuple, class, and dataclass.

- Sorting and attribute access are faster with tuples, but named tuples cannot have methods like classes. Methods have to be defined as normal functions.
  - Sorting tuples can be 10X faster than sorting classes containing the same data. [The difference starts being perceptible when dealing with several hundred instances.](https://www.youtube.com/watch?v=ybh0GttfM8o)
  - Named tuples are immutable, but they can contain mutable objects since the tuple just stores references to objects. The reference to a mutable object like a list does not change when it is mutated.
    - On the same token, those objects must be mutated in place since assignment is forbidden. 
    - The same caveats with mutable defaults in function definitions apply to named tuples, since the defaults are like class variables. It is generally advisable to use None or a non-mutable default, then explicitly declare mutable values even if they are empty. This will create an instance variable that can be mutated without affecting other instances. 
<br><br>
- Methods can be defined within classes which helps keep code organized logically. It also allows having methods for different classes to have the same name but different functionality or implementation, and makes it easier to track how a function is intended to be used.
  - Classes and class instances are both mutable and can be assigned attributes that were not defined in the class definition. This is because by default, attributes are stored in a dictionary.
<br><br>
- While dataclasses are described as "mutable named tuples with defaults", that's because they're really just "customizable pre-built classes". They also benefit from inheritance and can have their own methods. 
  - The most obvious difference compared to regular classes is how convenient they are to define, similar to named tuples. Adding the `@dataclass` decorator automatically adds `__init__`, `__repr__`, `__eq__`, and other optional methods like ordering or hashing. 
    - Any of these can be overridden by defining the method in the class definition, just like with normal class inheritance. 
  - Supplying mutable default values is performed using a default factory function, à la `field(default_factory=list)` rather than the `x = [] if x is None else x` idiom.
  - The dataclass module has tuple-like helper functions like `astuple()`, `asdict()`, and `replace()`. 
  - Generated comparison functions compare instances as if they were tuples.
  - `__iter__` can be defined to support iteration or unpacking.

If one does want to migrate from a named tuple to a class or data class, code that declares instances of named tuples or calls named attributes will work the same for classes without modification (assuming the relevant names are unchanged) because named tuples are classes themselves. However if functions are changed to methods one would have to change things like `func(instance.attribute)` to `instance.method`, and unpacking `*tuple` to `*astuple(dataclassinstance)`.


Notice how below, we need to provide the number of times each repeat is found in the sequence before creating an associated named tuple, even though this operation could be unnecessary or meaningless for the user. If we do that we might as well find all the start positions at the same time, since we need to iterate through the string to get the count anyway.

In [ ]:
from collections import namedtuple
from operator import attrgetter

repeattuple = namedtuple('RepeatTuple', 
                 names:=['sequence', 'length', 'starts', 'number'],
                 defaults=(None,)*(len(names)-1))

# Need to avoid conflicts with imported function names, one reason to use 
# import re instead of from re import findall.
def findrepeats(seq: str, length: int, inframe: bool=False) \
        -> list[repeattuple]:
    """
    Subequences that have overlapping, consecutive, or non-consecutive repeats.
    Sequences can be filtered by whether they are in frame. 
    Sequences that are repeated in frame will always be included,
    even if the sequence is also repeated out of frame."""
    s = '\S'                            # Match any non-whitespace characters.
    expr = re.compile(f'(?=(?P<repeat>{s}{{{length}}}))(?={s}+?(?P=repeat))')
    repeated = set(expr.findall(seq))
    
    def findall(repeat, seq, inframe):
        'Returns start positions indexed to 1.'
        starts, pos = [], 0
        while (pos := seq.find(repeat, pos)) != -1:
            pos += 1
            if (pos % 3 == 1 if inframe else True):
                starts.append(pos)
        return starts
    # repeated contains any repeat regardless of frame.
    # findall() checks whether copies of repeats are in frame.
    # If there are less than two in frame copies, 
    # the sequence is not repeated in frame.
    # Checking the inframe condition in a list comprehension is safer 
    # than doing these checks separately, since sets do not guarantee order.
    # An intermediate list of tuples matching just the repeat sequences to the 
    # results of findall is another solution with more duplication.
    # validstarts = [(repeat, findall(repeat, seq, inframe)) 
    #                for repeat in repeated]
    # validrepeats = [repeattuple(repeat, len(repeat), starts, num) 
    #                 for repeat, starts in validstarts 
    #                 if (num:=len(starts)) > 1]
    validrepeats = [repeattuple(repeat, len(repeat), starts, num) 
                    for repeat in repeated 
                    if (num:=len(starts:=findall(repeat, seq, inframe))) > 1]
    return sorted(validrepeats, key=attrgetter('starts'))

s6 = 'accghacagacjhacagfghacagacagact'

# 'hacag' only has one in frame copy at pos 13 so it is not repeated in frame.
# 'cagac' has an out of frame repeat but that repeat position is not reported 
# when inframe is True. 
# Unpacking list makes each element a separate argument so they can be printed 
# on separate lines.
print("All repeats:", *findrepeats(s6, 5), sep='\n')
print("\nOnly in frame repeats:", *findrepeats(s6, 5, True), sep='\n')

#### Dataclasses and fields

Initializing dataclass attributes that depend on other attributes works a little bit differently than it does within an `__init__` method. 
Variables listed at the top of a dataclass definition are actually interpreted as "fields", not instance variables. These are used to generate `__init__` and `__repr__`, and options passed to `fields()` regulate how each field is incorporated into these methods as well as others.

Below we have a field for the length of a string `self.length`, and we want to automatically calculate the length without the user having to type `len(str)`. This calculation cannot be included in the field definition because at that point `self.seq` has not been initialized. This would be like listing `self.length = len(self.seq)` as a parameter to `__init__`.

Instead `self.length` must be initialized in the `__post_init__` method. However, if we just do that without declaring `self.length` as a field, `self.length` will not be included in `__repr__`.

We could declare a default value for `self.length` up top then change its value in `__post_init__`, but then `length` would be a positional argument. This influences the order that we'd want to list the parameters and implies the user needs to supply a value. If `length` is listed as the second parameter, the user will have to either supply a value for `length` or name all the values following `length`. 

What we should do instead is declare the `self.length` field so it is listed by `__repr__` in the desired order, but exclude it from `__init__` to avoid confusing the user. Note that since `length` is not a parameter in `__init__`, it is not expected as a positional argument. As written below, the positional arguments to `RepeatDataClass` are `seq`, `number`, and `starts` *regardless of the position `length` is listed in*. 

In [118]:
from dataclasses import dataclass, field

# Setting @dataclass(order=True) implements comparison methods which 
# compare instances as if they are tuples, in order of their elements.
# Sorting by length is more useful than sorting by string value 
# (approximately alphabetical order).
# 
# The default repr() will list length first, but it would be better 
# to list the sequence in the repr first. 
# A custom repr can allow this and in this case is more convenient than
# writing out all the comparison methods. 
# Since self.length is defined last, it is last in __dict__.items(), and a 
# boilerplate repr listing these items in order will get the job done.
#   
@dataclass(order=True)
class RepeatDataClass:
    length: int = field(init=False)
    seq: str
    number: int = 0
    starts: list[int] = field(default_factory=list)

    # __post_init__ takes no arguments.
    def __post_init__(self):
        self.length = len(self.seq)

    # Comment out to see output of the default __repr__  
    def __repr__(self) -> str:
        repstr = '(' + ', '.join(f'{attr}={repr(val)}'
                                for attr, val in self.__dict__.items()) + ')'
        return self.__class__.__name__ + repstr

    # Nothing stopping us from defining methods
    # Here we do not need to worry about naming conflicts between functions 
    # from other modules.
    # That could be a concern when using the @staticmethod decorator, 
    # be sure to test that.
    def findall(self, seq, inframe=False):
        'Returns start positions indexed to 1.'
        starts, pos = [], 0
        while (pos := seq.find(self.seq, pos)) != -1:
            pos += 1
            if (pos % 3 == 1 if inframe else True):
                starts.append(pos)
        return starts
    def count(self, seq, inframe=False):
        'Use if locations are not needed. Otherwise len(starts) from where()can be used.'
        if inframe:
            count, pos = 0, 0
            while (pos := seq.find(self.seq, pos)) != -1:
                pos += 1
                if (pos % 3 == 1):
                    count += 1
            return count
        else:
            return seq.count(self.seq)

s6 = 'aghacagacjhgfcjhfghacagacagt'
p = RepeatDataClass('acaga',18,2)
q = RepeatDataClass('acaa')
print('p:',p,'\nq:', q)
print(f"Total number and locations of in-frame '{p.seq}' repeats in '{s6}':",
      f"{p.count(s6)}, {p.findall(s6)}")
print(f"Number and locations of in-frame '{p.seq}' repeats in '{s6}':",
      f"{p.count(s6,True)}, {p.findall(s6,True)}")


p: RepeatDataClass(seq='acaga', number=18, starts=2, length=5) 
q: RepeatDataClass(seq='acaa', number=0, starts=[], length=4)
Total number and locations of in-frame 'acaga' repeats in 'aghacagacjhgfcjhfghacagacagt': 2, [4, 20]
Number and locations of in-frame 'acaga' repeats in 'aghacagacjhgfcjhfghacagacagt': 1, [4]


In [91]:
q.newthing = []
q

RepeatDataClass(seq='gaca', number=0, starts=[])

In [47]:
class Repeat():
    def __init__(self, seq, start=None):
        self.seq = seq
        self.len = len(seq)
        self.start = start
    def __repr__(self) -> str:
        repstr = '(' + ', '.join(f'{attr}={repr(val)}'
                                for attr, val in self.__dict__.items()) + ')'
        return self.__class__.__name__ + repstr
    def __str__(self) -> str:
        return self.seq

    def where(self, seq, inframe=False):
        'Returns start positions indexed to 1.'
        starts, pos = [], 0
        while (pos := seq.find(self.seq, pos)) != -1:
            pos += 1
            if (pos % 3 == 1 if inframe else True):
                starts.append(pos)
        return starts
    def count(self, seq, inframe=False):
        'Use if locations are not needed. Otherwise len(starts) from where()can be used.'
        if inframe:
            count, pos = 0, 0
            while (pos := seq.find(self.seq, pos)) != -1:
                pos += 1
                if (pos % 3 == 1):
                    count += 1
            return count
        else:
            return seq.count(self.seq)
s6 = 'aghacagacjhgfcjhfghacagacagt'
r = Repeat('acaga')
r.count(s6, True)

1

In [63]:
r = repeattuple(length=2)
q = repeattuple(length=5)
q.starts.append(8)
q,r

(RepeatTuple(sequence='', length=5, starts=[8, 8, 8], number=0),
 RepeatTuple(sequence='', length=2, starts=[8, 8, 8], number=0))

In [ ]:
class Repeat():
    def __init__(self, seq, start=None):
        self.seq = seq
        self.len = len(seq)
        self.start = start
    def __repr__(self) -> str:
        repstr = '(' + ', '.join(f'{attr}={repr(val)}'
                                for attr, val in self.__dict__.items()) + ')'
        return self.__class__.__name__ + repstr
    def __str__(self) -> str:
        return self.seq

    def where(self, seq, inframe=False):
        'Returns start positions indexed to 1.'
        starts, pos = [], 0
        while (pos := seq.find(self.seq, pos)) != -1:
            pos += 1
            if (pos % 3 == 1 if inframe else True):
                starts.append(pos)
        return starts
    def count(self, seq, inframe=False):
        'Use if locations are not needed. Otherwise len(starts) from where()can be used.'
        count, pos = 0, 0
        while (pos := seq.find(self.seq, pos)) != -1:
            pos += 1
            if (pos % 3 == 1 if inframe else True):
                count += 1
        return count
s6 = 'agacagacjhgfcjhfgacagacagt'
r = Repeat('acaga')
r.count(s6, True)

In [ ]:
r.newthing = 9
r

In [ ]:
set(['aga', 'gac', 'aca', 'cag', 'aga', 'gac', 'cjh', 'cjh'])

In [ ]:
help(list)

In [ ]:

sorted([orf(sequence='ATGCGAATGGCTGATGCGGTAGTAATCTGA', start=6, end=36, length=30, readingframe=0), orf(sequence='ATGGCTGATGCGGTAGTAATCTGA', start=12, end=36, length=24, readingframe=0), orf(sequence='ATGCGGTAG', start=19, end=28, length=9, readingframe=1)], key=attrgetter('sequence'))

In [ ]:
%whos